In [13]:
import numpy as np
from scipy.stats import norm #easy way to access the CDF of a normal RV

In [14]:
# Practical problem 1 -- implements the desired formula for price of European call option.

def ECBS(T,S0,r,sigma,E):
    # Price of the Black--Scholes solution for a European call option given its price S0 at t=0.
    # Given the price at another time t>0, should be called with maturity time T-t to treat t as
    # the initial time.
    d1 = (1/(sigma*np.sqrt(T)))*(np.log(S0/E)+(r+(1/2)*(sigma**2)*T)) #d1 at t=0
    d2 = d1-sigma*np.sqrt(T) #d2 at t=0
    discount = E*np.exp(-r*T)    #discount factor at t=0                         
    return S0*norm.cdf(d1) - discount*norm.cdf(d2)

In [188]:
ECBS(1,10,.06,.1,9)

1.5429374445144521

In [16]:
# Practical problem 2 -- defines auxiliary functions, Monte Carlo sampling for European call option, 
# and collects the MC samples for various values of M into one experiment.

# Payoff function
def Payoff(x,E):
    return max(x-E,0)

# Asset price if the sampled normal is z
def AssetPrice(S0,r,sigma,T,z):
    return S0*np.exp((r-(1/2)*(sigma**2))*T+sigma*np.sqrt(T)*z)

# Discounted asset price -- if z is a standard normal this gives one Monte Carlo sample
def MonteCarloSample(S0,r,sigma,T,E,z):
    S=AssetPrice(S0,r,sigma,T,z)
    DiscountPayoff = np.exp(-r*T)*Payoff(S,E)
    return DiscountPayoff

# Monte Carlo sampling routine for M samples. Returns a tuple of samples.
def MonteCarlo(M,S0,r,sigma,T,E):
    NormalSamples = norm.rvs(size=M) #norm.rvs returns samples of normal RVs, standard by default
    Samples = []
    for j in NormalSamples:
        Samples.append(MonteCarloSample(S0,r,sigma,T,E,j))
    return Samples

# Experiment to explore the convergence of the Monte Carlo scheme. Returns averaged values and estimates
# for error for various quantities of samples.
def Experiment(k,S0,r,sigma,T,E):
    Exp = []
    for j in range(1,k):
        x=MonteCarlo(2**j,S0,r,sigma,T,E) #Monte Carlo scheme with 2^j samples.
        a=np.average(x)
        b=np.std(x,ddof=1) #std for standard deviation; ddof=1 is the '-1' to have an unbiased estimator of the variance
        Exp.append([a,1.96*b/np.sqrt(2**j)])
    return Exp

In [18]:
Experiment(22,10,.06,.1,1,9)

[[3.1764335520903666, 0.5252302757955041],
 [1.0515608878762899, 0.5424421961321659],
 [1.7146412895525271, 1.0073913082593413],
 [1.4020549385266787, 0.5806352910808793],
 [1.3393856731306921, 0.30883590104958625],
 [1.3492109235960021, 0.22500782110815237],
 [1.6856599353085058, 0.18177601536414198],
 [1.535765795177426, 0.114749158443375],
 [1.5075062130176198, 0.08477375045756916],
 [1.5704700403593526, 0.06082085693144876],
 [1.539026665125051, 0.04160994102102311],
 [1.5343180102596161, 0.029441126536872243],
 [1.538670827353186, 0.021047534071096313],
 [1.5377110837794281, 0.014660852679214808],
 [1.5478940550728828, 0.010462356341236287],
 [1.54400873880765, 0.00743459898348229],
 [1.5409270894517872, 0.00523018794540991],
 [1.543453619304037, 0.003706544695243285],
 [1.542600073562498, 0.0026146619421162183],
 [1.5415893203263265, 0.0018533490439755604],
 [1.5424828609695027, 0.0013100129686891384]]

In [187]:
# Practical 2, Problem 1, part one -- MC for up-and-out European call option, standard method.

r=.05
sigma=.25
B=9
S0=5
E=6
T=1
N=10**3 #Number of sampling times
times=[T/N for k in range(N)] #increments of time, all of same size

# Ingredientfor payoff function. Important: this is not the full payoff function, this just indicates how to
# create the array of values along a discretized sample path, and then to check the barrier 
# condition. For the full payoff need to also check against strike price and discounting.):
def UaOPayoff():
    Sa=S0
    Payoffs = [Sa]
    for time in times:
        PayoffIncrement = Sa*(np.exp((r-(1/2)*(sigma**2))*time+sigma*np.sqrt(time)*np.random.normal(0,1)))
        Payoffs.append(PayoffIncrement)
        Sa=PayoffIncrement
    if np.max(Payoffs) < B:
        return Payoffs[-1]
    else:
        return 0
    
# To complete Practical 2, problem 1 -- use this idea to create payoff function inside of the Monte Carlo method.
# For other options (up-and-in, other barrier options, Asian options, etc.) the same idea can be
# used by modifying the if-else to the situation at hand.